# **<center>EE475-Final Project: Machine Learning for NBA Prediction</center>**

**<center>Group Members: Haoning Zhang，Shuai Yue,Yujie Jiang,  Zhisheng Lin</center>**

---

## **1.Topic**
Our project focus on using the dataset of 2018-2019 NBA Seasons to predict the results of 2019-2020 NBA Regular Seasons

Basketball is one of the most competitive sports around the world. The National Basketball Association (NBA) is a men's professional basketball league in North America, which also gathers world’s top basketball athletes. In addition to enjoy the exciting matches, we also want to be able to predict which teams could make playoffs and which team will finally get the championship. As we can get enough data from basketball reference websites, this could be a very practical way to apply machine learning into the real world.

## **2.Design description**
1.   **Data Collection:**

      Collect the NBA seasons data from web:https://www.basketball-reference.com/leagues/NBA_2019.html. 
      The data consists of three datasets. One that is Team Per Game Stats, One that is Opponent Per Game Stats,and one
      that is Miscellaneous Stats.Then, we refine the data. The related function and detailed annotation can be found in function: 
          DataTrim(oppo_data,team_data,mis_data)
          Functions: Extract the feature of the data. Feature is described by using Elo rating system.
          Input: 
            1. merged_data: data merged for feature extraction.
            2. label_result：scores of each team, which would be processed to get a win or lose result.
          Output:
            features and labels for training.
2.   **Feature Extraction:**

      Use the elo score rating system to describe the feature.Elo score refer to the formula that can calculate each other's win expectation. There are four steps for getting features.

          2.1. GetEloScore(team):
            Functions: Read the Elo Score of the team
            Input: 
                  Team name, which would be used as the index to get the score
            Output:
                  Elo Score of the team.

          2.2.UpdateEloScore(merged_data,label_result):
            Functions: Update the Elo score of each team
            Input: 
                  1. merged_data: data merged for feature extraction.
                  2. label_result：scores of each team, which would be processed to get a win or lose result.
            Output:
                  features and labels for training.

          2.3.CalcEloScore(winteam, loseteam):
            Functions: Calculate the Elo Score of both team
            Input: 
                  Win and lose team's names, which would be used as the index to get the score
            Output:
                  Updated Elo Scores
        
          2.4.FeatureExtraction(merged_data,label_result):
            Functions: Extract the feature of the data. Feature is described by using Elo rating system.
            Input: 
                  1. merged_data: data merged for feature extraction.
                  2. label_result：scores of each team, which would be processed to get a win or lose result.
            Output:
                  features and labels for training.
 
## **3. Model Training**
We use various ways, like logistic regression, decision tree, etc to train data, getting our model used for predicting. The related function and detailed annotation can be found in function:

            clf = linear_model.LogisticRegression(solver='lbfgs')
            clf.fit(train_X,train_y)
            clf = GradientBoostingClassifier(n_estimators=50)
            clf.fit(train_X, train_y)
            clf = KNeighborsClassifier()
            clf.fit(train_X, train_y)
            clf = RandomForestClassifier(n_estimators=8)
            clf.fit(train_X, train_y)
            clf = tree.DecisionTreeClassifier()
            clf.fit(train_X, train_y)

Finally we choose Logistic Regression  to fit the model.
   
## **4. Result Prediction**
Generate the prediction data, and then predict.
      
## **5. Validation and Accuracy**
We use 10-fold cross-validation:
            
            cross_val_score(clf, train_X, train_y, cv = 10, scoring='accuracy', n_jobs=-1).mean()

## **6. Code**

See more in our youtube video: https://youtu.be/pHDeG6366FM

In [1]:
##########################################################################################
# EE475-Final Project: Machine Learning for NBA Prediction
# Author:Haoning Zhang，Shuai Yue, Yujie Jiang,  Zhisheng Lin
# Last Edit: 2019.12.09
# **This code is avaliable in Python
##########################################################################################

from __future__ import division
import math
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import pandas as pd 

from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import csv

In [2]:
def DataTrim(oppo_data,team_data,mis_data):
    '''
    Functions: 1. Drop unrelated data like name of the arena. 2. Merge three input array.
    Input: 
        1. oppo_data: Opponent_Per_Game_Stat;
        2. team_data: Team_Per_Game_Stat;
        3. mis_data: Miscellaneous_Stat;
    Output:
        new merged data.
    Tag Meaning:
        oppo_data(Opponent Per Game Data):
            Rk: Rank;  Team: Team Name;  G: Games;  MP: Minutes Played;  FG:Opponent Field Goals;
            FGA: Opponent Field Goal Attempts;   FG%:Opponent Field Goal Percentage;  
            3P: Opponent 3-Point Field Goals;    3PA: Opponent 3-Point Field Goals Attempts;  
            3P%: Opponent 3-Point Field Goal Percentage;
            2P: Opponent 2-Point Field Goals;    2PA: Opponent 2-Point Field Goals Attempts; 
            2P%: Opponent 2-Point Field Goal Percentage;
            FT: Opponent Free Throws;    FTA: Opponent Free Throws Attempt;   FT%: Opponent Free Throws Percentage;
            ORB: Opponent Offensive Rebounds;    DRB: Opponent Defensive Rebounds;
            TRB: Opponent Total Rebounds;        AST: Opponent Assists;                      STL:  Opponent Steals;    
            BLK: Opponent Blocks;      TOV: Opponent Turnovers;     PF: Opponent Personal Fouls;     PTS: Opponent points; 
        team_data(Team Per Game Data):
            Rk: Rank;  Team: Team Name;  G: Games;  MP: Minutes Played;  
            FG: Field Goals;                FGA:  Field Goal Attempts;           FG%: Field Goal Percentage;  
            3P:  3-Point Field Goals;       3PA:  3-Point Field Goals Attempts;  3P%:  3-Point Field Goal Percentage;
            2P:  2-Point Field Goals;       2PA:  2-Point Field Goals Attempts;  2P%:  2-Point Field Goal Percentage;
            FT:  Free Throws;               FTA:  Free Throws Attempt;           FT%: Free Throws Percentage;
            ORB:  Offensive Rebounds;       DRB: Defensive Rebounds;             TRB:  Total Rebounds;        
            AST: Assists;    STL: Steals;   BLK:  Blocks;   TOV: Turnovers;  PF: Personal Fouls;    PTS: points; 
        mis_data(Miscellaneous Data):
            Rk: Rank;  Team: Team Name;  Age:Average Age;  W: Wins;  L:Losses;  
            PW: Pythagorean Wins;  PL: Pythagorean Losses;  MOV: Margin of Victory;
            SOS: Strength of Schedule;  SRS: Simple Rating System;  ORtg: Offensive Rating;  DRtg Defensive Rating;
            NRtg: Net Rating;           Pace: Pace Factor;          FTr: Free Throw Attemp Rate;
            3PAr: 3-Point Attempt Rate; TS%: True Shooting Percentages;      eFG%: Effective Field Goal Percentage;
            TOV%: Turnover Percentage;  ORB%: Offensive Rebound Percentage;  FT/FGA: Free Throws Per Field Goal Attempt;
            eFG%: Opponent Effective Field Goal Percentage;                  TOV%: Opponent Turnover Percentage;
            DRB%: Defensive Rebound Percentage;     FT/FGA: Opponent Free Throws Per Field Goal Attempt; 
            Arena: Name of the Arena    Attend.: Attendance                  Attend./G: Attendance Per Home Game
    '''
    
    #Data Trim,drop unrelated data.
    oppo_data_trim=oppo_data.drop(['Rk','G','MP'],axis = 1)# Drop Rank, Games, Minutes Played
    team_data_trim=team_data.drop(['Rk','G','MP'],axis = 1)# Drop Rank, Games, Minutes Played
    mis_data_trim=mis_data.drop(['Rk', 'PW','PL','Arena'],axis = 1)# Drop Rank, Pythagorean Wins,Pythagorean Losses, Arena

    #Merge 3 kinds of data to one dataset
    merged_temp=pd.merge(oppo_data_trim, team_data_trim, how = 'left', on = 'Team')
    merged_data=pd.merge(merged_temp, mis_data_trim, how = 'left', on = 'Team')
    
    return merged_data.set_index('Team', drop = True, append = False)

def FeatureExtraction(merged_data,label_result):
    '''
    Functions: Extract the feature of the data. Feature is described by using Elo rating system.
    Input: 
        1. merged_data: data merged for feature extraction.
        2. label_result：scores of each team, which would be processed to get a win or lose result.
    Output:
        features and labels for training.
    '''
    train_X=[]
    train_y=[]
    
    for index, rows in label_result.iterrows():
        
        ## 1. read data from result.
        #visit_team means the visiting team, home team means the home team
        #_score means the score of both teams
        visit_team = rows['Visitor/Neutral']
        visit_team_score = rows['PTSV']
        home_team = rows['Home/Neutral']
        home_team_score = rows['PTSH']
        
        ## 2. Get the current elo score for each team.
        # this score will be updated after the match, which is how Elo score works.
        visit_team_elo=GetEloScore(visit_team)
        home_team_elo=GetEloScore(home_team)
        home_team_elo=home_team_elo+100 # for the home team, they have a home buff which adds 50 performance score
        
        ## 3. Create the feature list.
        # first element of the feature list would be the Elo score;
        # other elements would be the data collected from .csv file(like Ek,FGA,3P);
        # features of visiting team and home teams would be merged in one row;
        
        #input first element, elo score
        feature_visit_list=[visit_team_elo]
        feature_home_list=[home_team_elo]
        
        #feature list of the visiting team
        feature_visit_add=np.nan_to_num(merged_data.loc[visit_team])#add features from former merged data
        feature_visit_list.extend(feature_visit_add)#append features together
        
        #feature list of the home team
        feature_home_add=np.nan_to_num(merged_data.loc[home_team])
        feature_home_list.extend(feature_home_add)
        
        #feature list of both visiting and home teams
        feature_visit_home=feature_visit_list+feature_home_list
        

        ##4. Create the output dataset train_X and train_y.
        
        # save feature lists in one array, train_X
        try:
            train_X=np.vstack((train_X,feature_visit_home))
        except:
            train_X=feature_visit_home.copy()# if train_X is null, set the current feature as the first element.
        
        # save the results as label in one list, train_y. If home team wins, train_y=1
        game_result=np.int(visit_team_score>home_team_score)
        train_y.append(game_result)

        ##5. Update elo scores of the team.        
        # update elo scores of the team
        if visit_team_score>home_team_score:
            visit_team_elo_update, home_team_elo_update = CalcEloScore(visit_team, home_team)
        else:
            home_team_elo_update, visit_team_elo_update = CalcEloScore(home_team, visit_team)
            
        # save elo scores of the team    
        team_elos[visit_team] = visit_team_elo_update
        team_elos[home_team] = home_team_elo_update
    return train_X,train_y
    
    
def GeneratePredictFeature(pred_data,merged_data):
    '''
    Functions: Predict the probability for the visiting team to win the game.
    Input: 
        Team Schedule
    Output:
        Probability for the visiting team to win the game.
    '''  
    
    for index, rows in pred_data.iterrows():
        
        ## 1.Get the name of both visiting and home teams
        visit_team = rows['Visitor/Neutral']
        home_team = rows['Home/Neutral']
        
        ## 2. Get the current elo score for each team.
        visit_team_elo = GetEloScore(visit_team)
        home_team_elo = GetEloScore(home_team)
        
        ## 3. Create the feature list.
        # first feature would be the team Elo score
        feature_visit_list=[visit_team_elo]
        feature_home_list=[home_team_elo+100]
        
        #feature list of the visiting team
        feature_visit_add=np.nan_to_num(merged_data.loc[visit_team])#add features from former merged data
        feature_visit_list.extend(feature_visit_add)#append features together

        #feature list of the home team
        feature_home_add=np.nan_to_num(merged_data.loc[home_team])
        feature_home_list.extend(feature_home_add)
        
        #feature list of both visiting and home teams
        feature_visit_home=feature_visit_list+feature_home_list
        
        ##4. Create the output dataset train_X.    
        # save feature lists in one array, train_X
        try:
            pred_X=np.vstack((pred_X,feature_visit_home))
        except:
            pred_X=feature_visit_home.copy()# if pred_X is null, set the current feature as the first element.
        
    return pred_X


def CalcEloScore(winteam, loseteam):
    '''
    Functions: Calculate the Elo Score of both team
    Input: 
        Win and lose team's names, which would be used as the index to get the score
    Output:
        Updated Elo Scores.
    '''
    
    # get the current elo scores of both teams
    win_elo_score = GetEloScore(winteam)
    lose_elo_score = GetEloScore(loseteam)
    
    # calculate probability to win the match, refer to elo formula
    win_prob = 1/(1 + math.pow(10,(lose_elo_score - win_elo_score)/400))
    lose_prob = 1/(1 + math.pow(10,(win_elo_score - lose_elo_score)/400))
    
    # determine the coefficient K for different range of R
    if win_elo_score>=2400:
        K=16
    elif win_elo_score<=2100:
        K=32
    else:
        K=24
    
    # update elso score
    win_elo_score_new = round(win_elo_score + K*(1 - win_prob))
    lose_elo_score_new = round(lose_elo_score + K*(0 - lose_prob))
    return win_elo_score_new, lose_elo_score_new


def GetEloScore(team):
    '''
    Functions: Read the Elo Score of the team
    Input: 
        Team name, which would be used as the index to get the score
    Output:
        Elo Score of the team.
    '''
    
    try:
        return team_elos[team]
    except:
        team_elos[team] = init_elo
    return team_elos[team]



def UpdateEloScore(merged_data,label_result):
    '''
    Functions: Update the Elo score of each team
    Input: 
        1. merged_data: data merged for feature extraction.
        2. label_result：scores of each team, which would be processed to get a win or lose result.
    Output:
        features and labels for training.
    '''
    
    for index, rows in label_result.iterrows():
        
        ## 1. read data from result.
        #visit_team means the visiting team, home team means the home team
        #_score means the score of both teams
        visit_team = rows['Visitor/Neutral']
        visit_team_score = rows['PTSV']
        home_team = rows['Home/Neutral']
        home_team_score = rows['PTSH']
        
        ## 2. Get the current elo score for each team.
        # this score will be updated after the match, which is how Elo score works.
        visit_team_elo=GetEloScore(visit_team)
        home_team_elo=GetEloScore(home_team)
        home_team_elo=home_team_elo+100 # for the home team, they have a home buff which adds 50 performance score

        ##3. Update elo scores of the team.        
        # update elo scores of the team
        if visit_team_score>home_team_score:
            visit_team_elo_update, home_team_elo_update = CalcEloScore(visit_team, home_team)
        else:
            home_team_elo_update, visit_team_elo_update = CalcEloScore(home_team, visit_team)
            
        # save elo scores of the team    
        team_elos[visit_team] = visit_team_elo_update
        team_elos[home_team] = home_team_elo_update

In [3]:
if __name__ == '__main__':
    
    init_elo = 1600
    team_elos = {}
    
    ## 1.Data Collection. refer to:https://www.basketball-reference.com/leagues/NBA_2019.html
    oppo_data=pd.read_csv('new_data/train/18-19Opponent_Per_Game_Stat.csv')
    team_data=pd.read_csv('new_data/train/18-19Team_Per_Game_Stat.csv')
    mis_data=pd.read_csv('new_data/train/18-19Miscellaneous_Stat.csv')
    last_year_result = pd.read_csv( 'new_data/train/18-19TeamResult.csv')
    
    ## 2. Data Pre-processing
    trim_data=DataTrim(oppo_data, team_data, mis_data)
    
    ## 3. Feature Extraction
    print('Extracting features...')
    train_X,train_y=FeatureExtraction(trim_data,last_year_result)
    print('Done.')
    
    ## 4. Model Training. 
    # You can also choose different classifiers.
    print('Fitting on game samples...')
    clf = linear_model.LogisticRegression(solver='lbfgs')
    clf.fit(train_X,train_y)
#     clf = GradientBoostingClassifier(n_estimators=50)
#     clf.fit(train_X, train_y)
#     clf = KNeighborsClassifier()
#     clf.fit(train_X, train_y)
#     clf = RandomForestClassifier(n_estimators=8)
#     clf.fit(train_X, train_y)
#     clf = tree.DecisionTreeClassifier()
#     clf.fit(train_X, train_y)
    print('Fitting done.')
    
    ## 5. 10-fold Validation
    print("Doing cross-validation..")
    print('The accuracy is: ')
    print(cross_val_score(clf, train_X, train_y, cv = 10, scoring='accuracy', n_jobs=-1).mean())
    
    ## 6. Prediction     
    print('Predicting games...')
    # Set data of this new season as input features.
    oppo_data_pred=pd.read_csv('new_data/prediction/19-20Opponent_Per_Game_Stat.csv')
    team_data_pred=pd.read_csv('new_data/prediction/19-20Team_Per_Game_Stat.csv')
    mis_data_pred=pd.read_csv('new_data/prediction/19-20Miscellaneous_Stat.csv')
    this_year_result = pd.read_csv( 'new_data/prediction/19-20TeamResult.csv')
    pred_team=pd.read_csv('new_data/prediction/19-20Schedule.csv')
    trim_data_pred=DataTrim(oppo_data_pred, team_data_pred, mis_data_pred)
     
    # Because some player may change team, the elo score of each team should be updated.
    # This updating process is done using UpdateEloScore() function.
    # Input data will be games which are already played in this season.
    UpdateEloScore(trim_data_pred,this_year_result)
    
    # Predict the results for those games which are not played yet. 
    pred_X=GeneratePredictFeature(pred_team,trim_data_pred)
    pred_y=clf.predict(pred_X)
    print('Done.')

    ##7. Save the prediction result for the following games. Since Dec.9.2019
    # "1" means that the visiting team would win the game.
    # "0" means the home team would win.
    # Enjoy the game!
    team_schedule=pd.read_csv('new_data/prediction/19-20Schedule.csv')
    team_schedule.insert(2,'GameResult',np.array([pred_y]).T)
    team_schedule.to_csv('new_data/prediction/19-20SchedulePredictionResult.csv', mode='w', header=True, index=False)
    print(team_schedule)

Extracting features...
Done.
Fitting on game samples...
Fitting done.
Doing cross-validation..
The accuracy is: 
0.6471826323234147
Predicting games...
Done.
            Visitor/Neutral            Home/Neutral  GameResult
0      Los Angeles Clippers          Indiana Pacers           1
1       Cleveland Cavaliers          Boston Celtics           0
2           Toronto Raptors           Chicago Bulls           1
3          Sacramento Kings         Houston Rockets           0
4             Orlando Magic         Milwaukee Bucks           0
5           Detroit Pistons    New Orleans Pelicans           1
6    Minnesota Timberwolves            Phoenix Suns           1
7     Oklahoma City Thunder               Utah Jazz           0
8         Memphis Grizzlies   Golden State Warriors           0
9        Washington Wizards       Charlotte Hornets           0
10            Atlanta Hawks              Miami Heat           0
11           Denver Nuggets      Philadelphia 76ers           1
12        